In [132]:
import datasmoothie
import os
import tally
import json
import sys
ds = tally.DataSet(api_key=os.environ.get("TALLY_KEY"))

In [133]:
project_name = "tally_example_project"
dashboard_api = os.environ.get("DATASMOOTHIE_KEY")
banner_variables = []
stub_variables = []
dashboard_filters = []


In [ ]:
if len(dashboard_api) == 0:
    sys.exit(0)

In [134]:
ds.use_quantipy(f"../data/4_{project_name}.json",f"../data/4_{project_name}.csv")

In [135]:
dashboard_client = datasmoothie.Client(api_key=dashboard_api)

## Upload the data to dashboard

In [136]:
available_datasources = dashboard_client.list_datasources()
sources_with_project_name = [i for i in available_datasources['results'] if i['name'] == project_name]
if len(sources_with_project_name) == 0:
    print('create datasource')
    source = dashboard_client.create_datasource(project_name)
    source.update_meta_and_data(json.loads(ds.qp_meta), ds.qp_data)
else:
    print('fetch datasource')
    source = dashboard_client.get_datasource(sources_with_project_name[0]['pk'])


fetch datasource


In [137]:

reports = [i for i in dashboard_client.list_reports()['results'] if i['title'] == project_name]
if len(reports) == 0:
    print('create report')
    report = dashboard_client.create_report(project_name)
else:
    print('fetch report')
    report = dashboard_client.get_report(reports[0]['pk'])
report.update_content([])
report.elements = []

fetch report


In [138]:
report.elements

[]

In [139]:
variables = ds.variables()

In [140]:
if len(banner_variables) == 0:
  with open('../specs/common_variable_names.json', 'r') as f:
      common_variables = json.load(f)
  common_variables = [i.lower() for i in common_variables]
  for var in variables['single']:
    if any(match in var for match in common_variables):
      banner_variables.append(var)

In [141]:
if len(stub_variables) == 0:
  stub_variables = variables['single']
  stub_variables = [i for i in stub_variables if i not in banner_variables]

In [142]:
x_y_pairs = [(i, '@') for i in stub_variables]


In [143]:
report.add_charts(datasource_primary_key=source.get_id(),
                     x_y_pairs=x_y_pairs,
#                     user_filters=banner_variables,
                     comparison_variables=None,
                     charts_per_row=1)

In [144]:
if dashboard_filters is not None:
    if len(dashboard_filters) == 0:
        dashboard_filters = banner_variables
    report.update_meta_element('filter_type', 'global')
    report.update_meta_element('global_filter', "_&&_".join(dashboard_filters))

In [149]:
with open(f"../outputs/{project_name}_online_dashboard.txt", "w") as text_file:
    text_file.write("Link to report: %s" % report.get_url())
